In [1]:
#Librerías
import requests
import pandas as pd
import altair as alt

In [2]:
url="https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/628222/es/0700/false/BIE/2.0/[AQUÍ_VA_TU_TOKEN]?type=json"

In [3]:
#Respuesta de la consulta a la API  
respuesta = requests.get(url).json()

In [4]:
respuesta.keys()

dict_keys(['Header', 'Series'])

In [5]:
inpc=respuesta["Series"]
inpc=pd.DataFrame(inpc)
inpc = inpc["OBSERVATIONS"].tolist()
#Hacer dataframe únicamente con la lista OBSERVATIONS
inpc=pd.DataFrame.from_records(inpc[0])
inpc.columns= inpc.columns.str.lower()
#Selección de variables
inpc= inpc[["time_period", "obs_value"]]
inpc["time_period"]=pd.to_datetime(inpc["time_period"],format="%Y/%m/%d")
inpc["obs_value"]=pd.to_numeric(inpc["obs_value"])
inpc.sort_values(by=['time_period'], inplace=True)



In [6]:
#Calcular inflación
inpc["inflac"]=inpc["obs_value"].pct_change(periods=24)*100
#Dejar últimos 20 años
inpc.query('time_period>= "2012-01-01"', inplace = True)
inpc

,time_period,obs_value,inflac
236,2012-01-01,78.219845,3.939146
235,2012-01-02,78.466254,4.154285
234,2012-02-01,78.520344,4.012419
233,2012-02-02,78.483533,3.724186
232,2012-03-01,78.521095,3.723367
...,...,...,...
4,2021-09-01,114.482000,5.872452
3,2021-09-02,114.720000,6.126905
2,2021-10-01,115.340000,6.122224
1,2021-10-02,115.781000,6.355753


In [56]:
#Título
plot_title = alt.TitleParams("Índice Nacional de Precios al Consumidor", subtitle="Variación porcentual anual por quincena")
linea=alt.Chart(inpc,title=plot_title).mark_line(color="#285c4d",
    size=3).encode(
    x=alt.X('time_period:T',
    axis=alt.Axis(title="Fecha")),
    y=alt.Y('inflac',axis=alt.Axis(title="Var.%"))).properties(width=700, height=300)
    
alt.concat(linea).properties(
    title=alt.TitleParams(
        ['Fuente: @claudiodanielpc con información de INEGI. Índice Nacional de Precios al Consumidor.'],
        baseline='bottom',
        orient='bottom',
        anchor='start',
        fontWeight='normal',
        fontSize=8))


alt.ConcatChart(...)